# Exploratory Data Analysis - Energy Data

**Purpose**: Comprehensive exploratory analysis of energy data to identify patterns, anomalies, and data quality issues

**Date**: January 12, 2026

## Objectives
1. Analyze data distributions and patterns
2. Identify anomalies and outliers
3. Detect data quality issues (missing values, inconsistencies)
4. Examine year-over-year changes and trends
5. Investigate specific issues (2017-2018 anomalies)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Database connection
DB_CONFIG = {
    'host': '172.18.0.1',
    'port': 5432,
    'database': 'lianel_energy',
    'user': 'airflow',
    'password': 'P9xK2mN7vQ4wR8tY3sL6hJ5nB1cV0zX'
}

connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

print("✅ Database connection established")

## 2. Load ML Dataset Data

In [ ]:
# Load ML forecasting dataset
query = """
SELECT 
    cntr_code,
    year,
    total_energy_gwh,
    renewable_energy_gwh,
    fossil_energy_gwh,
    pct_renewable,
    pct_fossil,
    yoy_change_total_energy_pct,
    yoy_change_renewable_pct,
    energy_density_gwh_per_km2,
    area_km2
FROM ml_dataset_forecasting_v1
ORDER BY cntr_code, year
"""

df = pd.read_sql(query, engine)
print(f"✅ Loaded {len(df)} records")
print(f"Countries: {df['cntr_code'].nunique()}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
df.head()

## 3. Investigate 2017-2018 Anomalies

### 3.1 Year-over-Year Change Analysis (2018 Issue)

In [ ]:
# Focus on 2016-2019 period
df_2016_2019 = df[df['year'].between(2016, 2019)].copy()

# Calculate YoY changes manually for verification
df_2016_2019 = df_2016_2019.sort_values(['cntr_code', 'year'])
df_2016_2019['prev_year_total'] = df_2016_2019.groupby('cntr_code')['total_energy_gwh'].shift(1)
df_2016_2019['manual_yoy_pct'] = ((df_2016_2019['total_energy_gwh'] - df_2016_2019['prev_year_total']) / df_2016_2019['prev_year_total'] * 100)

# Identify extreme YoY changes in 2018
extreme_yoy = df_2016_2019[
    (df_2016_2019['year'] == 2018) & 
    (df_2016_2019['yoy_change_total_energy_pct'].abs() > 100)
].sort_values('yoy_change_total_energy_pct', ascending=False)

print("🔍 Countries with extreme YoY changes in 2018 (>100%):")
print(extreme_yoy[['cntr_code', 'year', 'total_energy_gwh', 'prev_year_total', 
                   'yoy_change_total_energy_pct', 'pct_renewable']].to_string())

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Total energy over time for countries with extreme changes
ax1 = axes[0]
for country in extreme_yoy['cntr_code'].head(10):
    country_data = df_2016_2019[df_2016_2019['cntr_code'] == country]
    ax1.plot(country_data['year'], country_data['total_energy_gwh'], 
             marker='o', label=country, linewidth=2)
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Energy (GWh)')
ax1.set_title('Total Energy Consumption - Countries with Extreme 2018 YoY Changes')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: YoY change percentage
ax2 = axes[1]
yoy_2018 = df_2016_2019[df_2016_2019['year'] == 2018].sort_values('yoy_change_total_energy_pct', ascending=False)
ax2.barh(yoy_2018['cntr_code'].head(15), yoy_2018['yoy_change_total_energy_pct'].head(15))
ax2.set_xlabel('YoY Change (%)')
ax2.set_title('Year-over-Year Change in 2018')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 3.2 Renewable Energy Percentage Analysis (2017 Issue)

In [ ]:
# Check renewable percentage patterns
renewable_analysis = df_2016_2019.groupby('year').agg({
    'pct_renewable': ['mean', 'min', 'max', 'std'],
    'cntr_code': 'count'
}).round(2)

print("📊 Renewable Energy Percentage Statistics by Year:")
print(renewable_analysis)

# Identify countries with 100% renewable in 2016-2017
high_renewable_2017 = df_2016_2019[
    (df_2016_2019['year'] == 2017) & 
    (df_2016_2019['pct_renewable'] >= 95)
].sort_values('pct_renewable', ascending=False)

print(f"\n⚠️ Countries with ≥95% renewable in 2017: {len(high_renewable_2017)}")
print(high_renewable_2017[['cntr_code', 'year', 'pct_renewable', 'renewable_energy_gwh', 
                           'fossil_energy_gwh', 'total_energy_gwh']].head(15).to_string())

# Visualize renewable percentage trends
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Average renewable percentage by year
ax1 = axes[0]
yearly_avg = df_2016_2019.groupby('year')['pct_renewable'].mean()
yearly_std = df_2016_2019.groupby('year')['pct_renewable'].std()
ax1.plot(yearly_avg.index, yearly_avg.values, marker='o', linewidth=2, markersize=8, label='Mean')
ax1.fill_between(yearly_avg.index, yearly_avg - yearly_std, yearly_avg + yearly_std, alpha=0.3)
ax1.set_xlabel('Year')
ax1.set_ylabel('Renewable Energy Percentage')
ax1.set_title('Average Renewable Energy Percentage by Year (with std dev)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution of renewable percentage by year
ax2 = axes[1]
for year in [2016, 2017, 2018, 2019]:
    year_data = df_2016_2019[df_2016_2019['year'] == year]['pct_renewable']
    ax2.hist(year_data, alpha=0.6, label=f'{year}', bins=20)
ax2.set_xlabel('Renewable Energy Percentage')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Renewable Energy Percentage by Year')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 Root Cause Analysis: Data Completeness by Year

In [ ]:
# Check raw data completeness
raw_data_query = """
SELECT 
    e.country_code,
    e.year,
    COUNT(DISTINCT e.product_code) as product_count,
    COUNT(*) as total_records,
    SUM(CASE WHEN p.fossil_flag = TRUE THEN e.value_gwh ELSE 0 END) as fossil_gwh,
    SUM(CASE WHEN p.renewable_flag = TRUE THEN e.value_gwh ELSE 0 END) as renewable_gwh,
    SUM(e.value_gwh) as total_gwh
FROM fact_energy_annual e
LEFT JOIN dim_energy_product p ON e.product_code = p.product_code
WHERE e.year IN (2016, 2017, 2018, 2019)
  AND e.harmonisation_version IS NOT NULL
  AND e.value_gwh > 0
GROUP BY e.country_code, e.year
ORDER BY e.country_code, e.year
"""

raw_data = pd.read_sql(raw_data_query, engine)

# Merge with ML dataset for comparison
comparison = df_2016_2019.merge(
    raw_data[['country_code', 'year', 'fossil_gwh', 'renewable_gwh', 'total_gwh']],
    left_on=['cntr_code', 'year'],
    right_on=['country_code', 'year'],
    suffixes=('_ml', '_raw')
)

# Calculate differences
comparison['fossil_diff'] = comparison['fossil_energy_gwh'] - comparison['fossil_gwh']
comparison['renewable_diff'] = comparison['renewable_energy_gwh'] - comparison['renewable_gwh']
comparison['total_diff'] = comparison['total_energy_gwh'] - comparison['total_gwh']

# Identify years with missing fossil data
missing_fossil = comparison[
    (comparison['fossil_energy_gwh'] == 0) & 
    (comparison['year'].isin([2016, 2017]))
].groupby('year').agg({
    'cntr_code': 'count',
    'fossil_energy_gwh': 'sum',
    'total_energy_gwh': 'mean'
}).round(2)

print("🔍 Data Completeness Analysis:")
print("\nCountries with zero fossil energy by year:")
print(missing_fossil)

# Summary statistics by year
yearly_summary = comparison.groupby('year').agg({
    'fossil_energy_gwh': ['mean', 'sum', lambda x: (x == 0).sum()],
    'renewable_energy_gwh': ['mean', 'sum'],
    'total_energy_gwh': 'mean'
}).round(2)

print("\n📊 Summary Statistics by Year:")
print(yearly_summary)

# Visualize fossil vs renewable by year
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Fossil energy by year
ax1 = axes[0, 0]
fossil_by_year = comparison.groupby('year')['fossil_energy_gwh'].sum()
ax1.bar(fossil_by_year.index, fossil_by_year.values, color='red', alpha=0.7)
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Fossil Energy (GWh)')
ax1.set_title('Total Fossil Energy by Year')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Renewable energy by year
ax2 = axes[0, 1]
renewable_by_year = comparison.groupby('year')['renewable_energy_gwh'].sum()
ax2.bar(renewable_by_year.index, renewable_by_year.values, color='green', alpha=0.7)
ax2.set_xlabel('Year')
ax2.set_ylabel('Total Renewable Energy (GWh)')
ax2.set_title('Total Renewable Energy by Year')
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Countries with zero fossil by year
ax3 = axes[1, 0]
zero_fossil_by_year = comparison[comparison['fossil_energy_gwh'] == 0].groupby('year')['cntr_code'].count()
ax3.bar(zero_fossil_by_year.index, zero_fossil_by_year.values, color='orange', alpha=0.7)
ax3.set_xlabel('Year')
ax3.set_ylabel('Number of Countries')
ax3.set_title('Countries with Zero Fossil Energy by Year')
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Average total energy by year
ax4 = axes[1, 1]
avg_total_by_year = comparison.groupby('year')['total_energy_gwh'].mean()
ax4.plot(avg_total_by_year.index, avg_total_by_year.values, marker='o', linewidth=2, markersize=8)
ax4.set_xlabel('Year')
ax4.set_ylabel('Average Total Energy (GWh)')
ax4.set_title('Average Total Energy Consumption by Year')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Analysis complete. Key findings:")
print(f"  - 2016-2017: {len(comparison[(comparison['year'].isin([2016, 2017])) & (comparison['fossil_energy_gwh'] == 0)])} country-year combinations with zero fossil energy")
print(f"  - 2018-2019: {len(comparison[(comparison['year'].isin([2018, 2019])) & (comparison['fossil_energy_gwh'] == 0)])} country-year combinations with zero fossil energy")